In [1]:
import time
import torch.backends.cudnn as cudnn
from torch import nn
from torch.utils.data import DataLoader
from models import Generator, Discriminator, TruncatedVGG19
#from datasets import SRDataset
from utils import *
from utils_.dataloader import Dataset as dataset
import utils_.helper_functions as helper_functions
from utils_.losses import calculate_metrics
import torch
from datetime import datetime
import wandb

import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

C:\Users\accou\anaconda3\envs\geo_env_n6\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


## get dataloader

In [2]:
def load_dataloader(dl_type="normal",sen2_test_tile="all"):
    
    working_directory = "C:\\Users\\accou\\Documents\\GitHub\\a-PyTorch-Tutorial-to-Super-Resolution\\"
    folder_path = "C:\\Users\\accou\\Documents\\thesis\\data_v2\\"
    dataset_file = "C:\\Users\\accou\\Documents\\thesis\\data_v2\\final_dataset.pkl"
    
    #filter_point=(367500.0,6822750.0) # Good one
    #filter_point=(292500.0,6810750.0) # okay
    #filter_point = (240750.0 , 6840000.0) # best one
    filter_point="Area"

    
    if dl_type == "normal":
        from utils_.dataloader import Dataset as dataset
        dataset_test   = dataset(folder_path,dataset_file,test_train_val="test",transform="histogram_matching",sen2_amount=1, sen2_tile = sen2_test_tile,   location="local",filter_point=filter_point)
        loader_test = DataLoader(dataset_test,batch_size=1, shuffle=False, num_workers=0,pin_memory=True,drop_last=False)
    if dl_type == "srcnn":
        from utils_.dataloader_srcnn import Dataset as dataset
        dataset_test   = dataset(folder_path,dataset_file,test_train_val="test",transform="interpolate",sen2_amount=1, sen2_tile = sen2_test_tile,   location="local",filter_point=filter_point)
        loader_test = DataLoader(dataset_test,batch_size=1, shuffle=False, num_workers=0,pin_memory=True,drop_last=False)
    if dl_type == "temporal":
        from utils_.dataloader_temporal import Dataset as dataset
        dataset_test   = dataset(folder_path,dataset_file,test_train_val="test",transform=None,sen2_amount=4, sen2_tile = sen2_test_tile,   location="local",filter_point=filter_point)
        loader_test = DataLoader(dataset_test,batch_size=2, shuffle=False, num_workers=0,pin_memory=True,drop_last=False)
    return loader_test

In [3]:
#srcnn_checkpoint = "model_files/srcnn_x4.pth"
#rcan_checkpoint  = "model_files/rcan_4x.pt"
srgan_checkpoint = "model_files/srgan_4x.pth.tar"
#srgan_temporal_checkpoint = "model_files/srgan_4x_temporal.tar"
#srgan_fusion_checkpoint = "model_files/srgan_4x_fusion.pth"

loader_SRGAN = load_dataloader("normal")

filtering for inference Area


In [4]:
load_from = "C:\\Users\\accou\\Documents\\GitHub\\a-PyTorch-Tutorial-to-Super-Resolution\\checkpoints\\checkpoint_SRGAN_25-04-2022_13-03-18_epoch61.pth.tar"

g2 = torch.load(srgan_checkpoint)["generator"]
g2 = g2.to(device)
g2 = g2.eval()

In [10]:
def save_georef(image,coor,path,prefix="im_"):
    import rasterio
    from rasterio.transform import from_origin
    
    image = image.to(device)
    image = image[0].cpu().detach().numpy()
    image = np.float32(image)
    #image = helper_functions.minmax_percentile(image)

    transform = from_origin(coor[0].item()-373,coor[1].item()+373, 2.5, 2.5) # WARNING: OFFSET introduced by dataloader sen2 tile extraction
    #print(image.shape,type(image))
    new_dataset = rasterio.open(path+prefix+str(coor[0].item())+"_y"+str(coor[1].item())+".tif", 'w', driver='GTiff',
                                height = 300, width = 300,
                                count=3, dtype=str(image.dtype),
                                crs=2154,
                                transform=transform)
    new_dataset.write(image)
    new_dataset.close()


def plot_image(image):
    image = image[0].detach().numpy()
    image = np.float32(image)
    image = np.transpose(image,(1,2,0))
    #image = helper_functions.minmax_percentile(image)
    plt.imshow(image)
    plt.show()


In [11]:
path_spot6 = "result/sen2/"
path_sen2 = "result/spot6/"
path_sr = "result/sr/"

In [12]:
count = 0
for item in loader_SRGAN:
    
    lr,hr,coor = item
    lr,hr = lr.to(device),hr.to(device)
    with torch.no_grad():
        sr = g2(lr)
    sr = convert_image(sr, source='[-1, 1]', target='imagenet-norm')
    
    save_georef(hr,coor,path_spot6,prefix="hr_")
    save_georef(lr,coor,path_sen2,prefix="lr_")
    save_georef(sr,coor,path_sr,prefix="sr_")
    #plot_image(sr)
    
    count = count+1
    if count%100==0:
        print(count,"/",len(loader_SRGAN),end="\r")

In [8]:
def bands_ind(sr):
    sr = sr[0]
    sr = sr.numpy()
    sr[0],sr[1],sr[2] = sr[0],helper_functions.minmax_percentile(sr[1]),helper_functions.minmax_percentile(sr[2])
    sr = np.transpose(sr,(1,2,0))
    plt.imshow(sr)


In [9]:
"""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
sr = sr.to(device)
helper_functions.plot_tensors_window(sr,sr,sr,fig_path="show")
"""

'\ndevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\nsr = sr.to(device)\nhelper_functions.plot_tensors_window(sr,sr,sr,fig_path="show")\n'